In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from unet.model import *
from unet.data import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard


Train Unet with EAT Data
----

### Train with data generator

In [4]:
data_gen_args = dict(rotation_range=0.2,
                     width_shift_range=0.05,
                     height_shift_range=0.05,
                     shear_range=0.05,
                     zoom_range=0.05,
                     horizontal_flip=False,
                     fill_mode='nearest')
generator = trainGenerator(8,'../Training Data','AllImages','AllMasks',data_gen_args,save_to_dir = None, image_color_mode = 'rgb')
model = unet()
#model.load_weights('unet_EAT_16.hdf5')
callbacks = [
    ModelCheckpoint("unet_EAT.hdf5"),
    ReduceLROnPlateau(monitor='loss', factor=0.1, patience=4),
    CSVLogger("data.csv"),
    TensorBoard(),
    EarlyStopping(monitor='loss', patience=10, restore_best_weights=False)
]
model.fit_generator(generator,steps_per_epoch=90,epochs=300,callbacks=callbacks)



<ipython-input-4-4d3123d2a93c>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator,steps_per_epoch=90,epochs=300,callbacks=callbacks)


Found 2123 images belonging to 1 classes.
Found 2123 images belonging to 1 classes.
Epoch 1/300
90/90 [==============================] - 34s 355ms/step - loss: 0.1158 - acc: 0.9587 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - iou: 0.1833 - lr: 1.0000e-04
Epoch 2/300
90/90 [==============================] - 32s 355ms/step - loss: 0.0526 - acc: 0.9669 - recall_1: 0.1351 - precision_1: 0.8387 - iou: 0.3047 - lr: 1.0000e-04
Epoch 3/300
52/90 [================>.............] - ETA: 12s - loss: 0.0516 - acc: 0.9829 - recall_1: 0.7263 - precision_1: 0.8297 - iou: 0.3407

Test Model
----

In [1]:
def generateTestsFromPath(test_path, target_size = (256,256)):
    images = os.listdir(test_path)
    for i in images:
        img = io.imread(test_path +"/" +i, as_gray = False)
        img = img / 255
        img = trans.resize(img,(256,256))
        img = img[:,:,0:3]
        img = np.reshape(img,(1,)+img.shape)
        yield img

def saveResults(results,save_path):
    shape = results.shape
    for i in range(0,shape[0]):
        result = results[i,:,:,0] * ((2**8)-1)
        resultRGB = np.zeros((shape[1],shape[2],3))
        resultRGB[:,:,0] = result
        resultRGB[:,:,1] = result
        resultRGB[:,:,2] = result
        matplotlib.image.imsave(os.path.join(save_path,"%d_predict.png"%i), resultRGB.astype(np.uint8))

gen = generateTestsFromPath("../Test Data/AllImages")
model = unet()
model.load_weights("unet_EAT.hdf5")
results = model.predict_generator(gen,verbose=1)
saveResults(results, "../Test Data/AllMasks")

NameError: name 'unet' is not defined

In [12]:
shape = results.shape
result = results[0,:,:,0] * ((2**8)-1)
resultRGB = np.zeros((shape[1],shape[2],3))
resultRGB[:,:,0] = result
resultRGB[:,:,1] = result
resultRGB[:,:,2] = result
resultRGB.shape

(256, 256, 3)

In [13]:
import matplotlib.pylab as plt
import skimage.io as io

model = unet()
model.load_weights("unet_EAT.hdf5")
test_path = "../Test Data/AllImages/15.png"

img = io.imread(test_path,as_gray = False)
img = img / 255
img = trans.resize(img,(256,256))
img = img[:,:,0:3]
img = np.reshape(img,(1,)+img.shape)
result = model.predict(img)

In [11]:
plt.imshow(result[0,:,:,0])
plt.show()

IndexError: too many indices for array: array is 2-dimensional, but 4 were indexed